In [1]:
import sys

sys.path.append("..")

import torch._dynamo
torch._dynamo.reset()

In [ ]:
import torch
import time
import matplotlib.pyplot as plt
from model.long import LongConfig, LongForCausalLM
from transformers import GPT2Config, GPT2LMHeadModel
from transformers import AutoTokenizer
from model import model_loader

def benchmark():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Benchmarking on {device}...")

    # # 1. Setup Your Model (Linear Attention)
    # my_config = LongConfig(
    #     vocab_size=50304,
    #     hidden_size=768,
    #     num_hidden_layers=18, # Increased depth
    #     num_heads=12,
    #     expansion_ratio=8/3,   # Ensures intermediate_size = 2048
    #     conv_kernel=4,
    #     hybrid_ratio=0         # Pure Linear Attention (fastest)
    # )
    # tokenizer = AutoTokenizer.from_pretrained("gpt2")

    # if tokenizer.pad_token is None:
    #     tokenizer.pad_token = tokenizer.eos_token
    #     tokenizer.pad_token_id = tokenizer.eos_token_id
    
    # my_model = LongForCausalLM(my_config).to(device)
    # my_model.config.pad_token_id = tokenizer.pad_token_id

    # # Compilation helps specifically with the scan operations
    # my_model = torch.compile(my_model, mode = "default")
    # my_model.eval()
    my_model, tokenizer = model_loader.get_model_and_tokenizer(
        model_type = "long",
        model_size = "small",
        device = "cuda"
    )

    # 2. Setup Standard Transformer (GPT-2)
    gpt_config = GPT2Config(
        vocab_size=50257,
        n_embd=256,
        n_layer=4,
        n_head=8,
        n_positions=2048
    )
    gpt_model = GPT2LMHeadModel(gpt_config).to(device)
    gpt_model.eval()

    # 3. The Race
    lengths = [100, 500, 1000, 2000]
    my_times = []
    gpt_times = []

    print("\n--- 🏎️ Starting Race (Generation Speed) ---")
    print(f"{'Seq Len':<10} | {'Yours (ms)':<15} | {'GPT-2 (ms)':<15} | {'Speedup':<10}")
    print("-" * 60)

    for seq_len in lengths:
        # Create a prompt of size 'seq_len'
        input_ids = torch.randint(0, 50000, (1, seq_len)).to(device)

        # --- NEW: Create Attention Mask ---
        # Since we have full valid data (no padding), the mask is all 1s.
        attention_mask = torch.ones_like(input_ids).to(device)
        
        # --- Benchmark Yours (Recurrent Step) ---
        # Warmup
        for _ in range(10):
            with torch.no_grad():
                _ = my_model(input_ids, attention_mask = attention_mask)
            
        torch.cuda.synchronize()
        start = time.time()
        
        # Generate 10 tokens
        # For your model, we simulate the Recurrent State update 10 times
        # (This is how you would use it in production)
        curr_input = input_ids[:, -1:]
        past_kv = None
        
        # Pre-fill state (the "prompt processing")
        with torch.no_grad():
             out = my_model(input_ids, attention_mask = attention_mask)
             past_kv = out.past_key_values
            

        # 2. Step-by-step Generation
        # For the step input, the mask is just [1] (valid token)
        step_mask = torch.ones((input_ids.shape[0], 1), dtype=torch.long, device=device)
        
        # Generate loop
        for _ in range(10):
            with torch.no_grad():
                out = my_model(curr_input, 
                               past_key_values=past_kv, 
                               attention_mask=step_mask)
                past_kv = out.past_key_values
                curr_input = torch.argmax(out.logits, dim=-1)

        torch.cuda.synchronize()
        my_time = (time.time() - start) * 1000 # ms
        my_times.append(my_time)

        # --- Benchmark GPT-2 (KV Cache) ---
        # GPT-2 uses KV caching, but attention matrix still grows
        with torch.no_grad():
             _ = gpt_model(input_ids, attention_mask=attention_mask)
             
        torch.cuda.synchronize()
        start = time.time()
        
        # We use generate() which handles KV cache efficiently
        with torch.no_grad():
            gpt_model.generate(
                input_ids, 
                attention_mask=attention_mask, # Pass mask!
                max_new_tokens=10, 
                do_sample=False,
                pad_token_id=tokenizer.pad_token_id # Explicitly pass pad token
            )
            
        torch.cuda.synchronize()
        gpt_time = (time.time() - start) * 1000
        gpt_times.append(gpt_time)

        print(f"{seq_len:<10} | {my_time:.2f}          | {gpt_time:.2f}          | {gpt_time/my_time:.2f}x")

    print("\n✅ Benchmark Complete.")

if __name__ == "__main__":
    benchmark()

Benchmarking on cuda...
[Model Loader] Identified Vocab Size: 50257
[Model Loader] Initializing Long LLM (small)...

--- 🏎️ Starting Race (Generation Speed) ---
Seq Len    | Yours (ms)      | GPT-2 (ms)      | Speedup   
------------------------------------------------------------


/workspace/long-attention/.venv/lib/python3.12/site-packages/torch/_inductor/compile_fx.py:236: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(
W0128 10:31:48.029000 49838 torch/_inductor/utils.py:1250] [0/0] Not enough SMs to use max_autotune_gemm mode


100        | 29865.10          | 112.45          | 0.00x
500        | 57.76          | 21.87          | 0.38x


In [ ]:
import torch
from model.long import LongConfig, LongForCausalLM  # Ensure this matches your file name

def count_parameters(model):
    """Counts valid trainable parameters."""
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def test_model_size():
    print("--- 🧪 Testing Model Configuration (Target: ~187M) ---")

    # 1. Instantiate Config
    config = LongConfig(
        vocab_size=50304,
        hidden_size=768,
        num_hidden_layers=18, # Increased depth
        num_heads=12,
        expansion_ratio=8/3,   # Ensures intermediate_size = 2048
        conv_kernel=4,
        hybrid_ratio=4         # Pure Linear Attention (fastest)
    )

    print(f"Config: L={config.num_hidden_layers}, H={config.hidden_size}, Vocab={config.vocab_size}")

    # 2. Instantiate Model
    print("Instantiating model... ", end="")
    try:
        model = LongForCausalLM(config)
        print("✅ Success!")
    except ImportError:
        print("\n❌ Error: Could not import LongForCausalLM. Make sure modeling_long.py is accessible.")
        return
    except Exception as e:
        print(f"\n❌ Error initializing model: {e}")
        return

    # 3. Calculate Parameters
    total_params = count_parameters(model)
    
    # Calculate Embedding vs Non-Embedding params
    # (Useful to know how much compute goes to "thinking" vs "looking up words")
    embed_params = config.vocab_size * config.hidden_size
    non_embed_params = total_params - embed_params

    print("\n--- 📊 Parameter Breakdown ---")
    print(f"Total Parameters:      {total_params:,} ({total_params/1e6:.2f}M)")
    print(f"Embedding Parameters:  {embed_params:,} ({embed_params/1e6:.2f}M)")
    print(f"Layer Parameters:      {non_embed_params:,} ({non_embed_params/1e6:.2f}M)")
    
    # 4. Verification
    target = 187_000_000
    diff = abs(total_params - target)
    print(f"\nDifference from 187M:  {diff:,} params")
    
    if 180_000_000 <= total_params <= 195_000_000:
        print("✅ Result: PERFECT MATCH for the 187M category.")
    else:
        print("⚠️ Result: Slightly off target (adjust layers/hidden_size if needed).")

if __name__ == "__main__":
    test_model_size()